In [1]:
# data  fragments from interim/2017/05/tab.csv

csv = """
	Год / Year	Кварталы / Quarters			
		I	II	III	IV
1. Сводные показатели / Aggregated indicators					
1.1. Валовой внутренний продукт1) / Gross domestic product1)					
Объем ВВП, млрд.рублей /GDP, bln rubles					
2010	46308	9996	10977	12086	13249
2011	59698	12844	14314	15663	16877
2012	66927	14925	16149	17442	18411
2013	71017	15892	17015	18543	19567
20142)	79200	17390	19128	20759	21923
20152)	83233	18568	19858	21967	22840
20162)	86044	18816	20430	22721	24077
2017		200913)			

Год / Year	Кварталы / Quarters	Янв. Jan.	Фев. Feb.	Март Mar.	Апр. Apr.	Май May	Июнь June	Июль July	Август Aug.	Сент. Sept.	Окт. Oct.	Нояб. Nov.	Дек. Dec.			
		I	II	III	IV												
1.2. Индекс промышленного производства1) / Industrial Production index1)																	
Индекс промышленного производства, 
в % к соответствующему периоду предыдущего года / percent of corresponding period of previous year																	
2015	99,2	99,9	98,3	99,5	99,1	100,0	98,2	101,2	98,2	97,6	99,1	98,5	100,2	99,7	98,4	101,0	98,1
2016	101,3	101,1	101,5	101,0	101,7	99,2	103,8	100,3	101,0	101,5	102,0	101,4	101,5	100,1	101,6	103,4	100,2
2017		100,1				102,3	97,3	100,8	102,3	105,6							
в % к предыдущему периоду / percent of previous period																	
2015		82,8	102,6	103,9	112,3	73,9	99,8	112,5	95,6	97,6	103,2	100,5	101,4	103,1	105,0	101,9	109,1
2016		84,4	103,1	103,3	113,1	74,7	104,4	108,8	96,3	98,1	103,8	99,9	101,5	101,7	106,6	103,6	105,8
2017		83,1				76,2	99,4	112,7	97,7	101,2							
период с начала отчетного года в % к соответствующему периоду предыдущего года / period from beginning of reporting year as percent of corresponding period of previous year																	
2015						100,0	99,1	99,9	99,4	99,1	99,1	99,0	99,1	99,2	99,1	99,3	99,2
2016						99,2	101,5	101,1	101,1	101,1	101,3	101,3	101,3	101,2	101,2	101,4	101,3
2017						102,3	99,7	100,1	100,7	101,7							
	Янв. Jan.	Фев. Feb.	Март Mar.	Апр. Apr.	Май May	Июнь June	Июль July	Август Aug.	Сент. Sept.	Окт. Oct.	Нояб. Nov.	Дек. Dec.
1.2.1. Индексы производства по видам деятельности1) (без исключения сезонности и фактора времени) / Industrial Production indices by Industry (without seasonal and time factor adjustment)												
"""

In [2]:

import pandas as pd

from csv2df.specification import Definition, Specification, SPEC
from csv2df.reader import Reader, open_csv
from csv2df.parser import extract_tables
from csv2df.emitter import Emitter

def get_dataframes(csvfile, spec=SPEC):
    tables = [t for csv_segment, pdef in Reader(csvfile, spec).items()
              for t in extract_tables(csv_segment, pdef)]
    # print(list(tables))
    
    emitter = Emitter(tables)
    dfa = emitter.get_dataframe(freq='a')
    dfq = emitter.get_dataframe(freq='q')
    dfm = emitter.get_dataframe(freq='m')
    return dfa, dfq, dfm

main = Definition(units={
                        'млрд.рублей': 'bln_rub', 
                        'период с начала отчетного года в % к соответствующему периоду предыдущего года': 'ytd',
                        'в % к соответствующему периоду предыдущего года': 'yoy',
                        'в % к предыдущему периоду': 'rog'
                        })

main.append(varname='GDP',
            text='Объем ВВП',
            required_units=['bln_rub'])

main.append(varname="INDPRO",
            text="Индекс промышленного производства",
            required_units=["yoy", "rog"],
            desc="Промышленное производство")


spec1 = Specification(default=main)

import io
csvfile1 = io.StringIO(csv )

In [3]:
import sys
# import traceback
import inspect
import linecache
       
def getline(filename, lineno):

    return linecache.getline(filename, lineno-1 )  # it kind of ignores file errors

    try:
        # line = linecache.getline(filename, lineno-1 )
        line = open(filename).readlines()[ lineno-1 ] # to notice file errors
    except FileNotFoundError as e:
        for path in ["<frozen importlib._bootstrap>", "<ipython-input-"]:
            if path in str(e):
                    break
        else:
            print ("FileNotFoundError", e)
                
    
    
start_frame = None

def format_inspect_stack(stack, start_frame, end_frame):
     # FrameInfo(frame, filename, lineno, function, code_context, index)
    calls = []
    for fi in stack:
        # line = open(fi.filename).readlines()[ fi.lineno-1 ]
        line = getline(fi.filename, fi.lineno )
        
        func_name = fi.frame.f_code.co_name
        calls.insert(0, f'{fi.filename}:{fi.lineno}:{func_name}: {line}' )
        if fi.frame is start_frame:
            break
        
    calls = ['  '*i + "-> " + line for i, line in enumerate( calls ) ] # add indents
    return ''.join( calls )
    
    
def trace_calls(frame, event, arg):

    co = frame.f_code
    func_name = co.co_name
    func_line_no = line_no = frame.f_lineno
    func_filename = filename = co.co_filename
    
    if func_name == 'trace_calls':
        return
        
    if event == 'return':
        return
        print ('%s => %s' % (func_name, arg))
        #if func_name == 'b':
        #    print( "LOCAL x", frame.f_locals['x'] )
        
    if event == 'call':
        
        caller = frame.f_back
        
        if caller is None:
            return 
        caller_line_no = caller.f_lineno
        caller_filename = caller.f_code.co_filename
        
        #line = linecache.getline(caller_filename, caller_line_no-1 )
        #if '_assert_has_no_duplicate_rows' in  line:
        #        print ("!!! CALL", line)
                
        if func_name == 'get_dataframes':
            global start_frame
            start_frame = caller
                        
        return trace_calls

    if event == 'line':
        
            line = getline( filename, line_no) 
        
            #check_tracked_info( frame )

            # triggered by some line fragment
            #if 'pd.DataFrame' in line:
            #    print ("!!", filename, line_no, "\n", line)
            
            # target:         df = df.pivot(columns='label', values='value', index='time_index') # in emitter.py:get_dataframe(..)
            if '# TRACE:' in  line:
                print ("!!!", line)
                

                print("inspect STACK\n")
                print(format_inspect_stack( inspect.stack()[1:], start_frame, end_frame=frame  )   )

                target_vars = line.split('# TRACE:')[-1].strip()
                #print("TRACEVAR:", target_vars, eval(target_vars, None, frame.f_locals ))
                target_vars = target_vars.split(',') 
                for target_var in map(str.strip, target_vars):
                    print("TRACEVAR:", target_var, frame.f_locals[target_var])


In [4]:


sys.settrace(trace_calls)
dfa1, dfq1, dfm1 = get_dataframes(csvfile1, spec1)




!!!         df = pd.DataFrame(self._collect(freq)) # TRACE: freq, df

inspect STACK

-> <ipython-input-4-3b1cdc9799f3>:4:<module>: sys.settrace(trace_calls)
  -> <ipython-input-2-e6621449485e>:15:get_dataframes:     emitter = Emitter(tables)
    -> /home/jurgis/dev/Freelancer/mini-kep/src/csv2df/emitter.py:149:get_dataframe:         df = pd.DataFrame(self._collect(freq)) # TRACE: freq, df

TRACEVAR: freq a
TRACEVAR: df   freq        label time_index    value  year
0    a  GDP_bln_rub 2010-12-31  46308.0  2010
1    a  GDP_bln_rub 2011-12-31  59698.0  2011
2    a  GDP_bln_rub 2012-12-31  66927.0  2012
3    a  GDP_bln_rub 2013-12-31  71017.0  2013
4    a  GDP_bln_rub 2014-12-31  79200.0  2014
5    a  GDP_bln_rub 2015-12-31  83233.0  2015
6    a  GDP_bln_rub 2016-12-31  86044.0  2016
7    a   INDPRO_yoy 2015-12-31     99.2  2015
8    a   INDPRO_yoy 2016-12-31    101.3  2016
!!!         df = pd.DataFrame(self._collect(freq)) # TRACE: freq, df

inspect STACK

-> <ipython-input-4-3b1cdc9799f3

In [5]:
dfa1

,year,GDP_bln_rub,INDPRO_yoy
time_index,,,
2010-12-31,2010,46308.0,NaN
2011-12-31,2011,59698.0,NaN
2012-12-31,2012,66927.0,NaN
2013-12-31,2013,71017.0,NaN
2014-12-31,2014,79200.0,NaN
2015-12-31,2015,83233.0,99.2
2016-12-31,2016,86044.0,101.3


In [6]:
print( dfa1 )

            year  GDP_bln_rub  INDPRO_yoy
time_index                               
2010-12-31  2010      46308.0         NaN
2011-12-31  2011      59698.0         NaN
2012-12-31  2012      66927.0         NaN
2013-12-31  2013      71017.0         NaN
2014-12-31  2014      79200.0         NaN
2015-12-31  2015      83233.0        99.2
2016-12-31  2016      86044.0       101.3
